<a href="https://colab.research.google.com/github/lamyse1/Data-Engineering-Projects/blob/main/week%203/DE_Week3_Exercise_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Integrating Weather Data Into a Sales Dataset Using APIs and MongoDB**


# Step 1: Extract Sales Data from CSV

In [23]:
import pandas as pd

# URL of the raw CSV file
url = 'https://raw.githubusercontent.com/DrManalJalloul/Introduction-to-Data-Engineering/main/sales_data.csv'

# Load the CSV data into a DataFrame
sales_data = pd.read_csv(url)

# Display the first few rows
print(sales_data.head())


         date product_id  sales_amount store_location
0  2025-02-05       P001           150       New York
1  2025-02-05       P002           300    Los Angeles
2  2025-02-05       P003           450        Chicago
3  2025-02-05       P004           600        Houston
4  2025-02-05       P005           750        Seattle


# Step 2: Fetch Weather Data from the API

In [24]:
pip install requests


In [25]:

# Function to fetch weather data for a given city and date
def fetch_weather_data(city, date, api_key):
    # Construct the API URL using the city and date
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}/{date}?unitGroup=metric&elements=datetime%2Cname%2Ctemp%2Chumidity%2Cdescription&include=days&key={api_key}&contentType=json"
    api_key = 'UBVEGBYAY6DNEE5TB8XX5CLD8'
    # Make the API request
    response = requests.get(url)
    data = response.json()

    # Extract weather information
    temp = data['days'][0]['temp']  # Temperature in Celsius
    humidity = data['days'][0]['humidity']  # Humidity percentage
    description = data['days'][0]['description']  # Weather description

    return temp, humidity, description

In [26]:
# Example usage
api_key = "UBVEGBYAY6DNEE5TB8XX5CLD8"
city = "New York"
date = "2023-09-01"

temp, humidity, description = fetch_weather_data(city, date, api_key)
print(f"Temp: {temp}, Humidity: {humidity}, Weather: {description}")

Temp: 20.0, Humidity: 49.7, Weather: Clear conditions throughout the day.


# Step 3: Combine Weather Data with Sales Data

In [27]:
# Iterate over each row in the sales_data DataFrame, call the weather function, and update the DataFrame
for index, row in sales_data.iterrows():
    temp, humidity, description = fetch_weather_data(row["store_location"], row["date"], api_key)

    # Save retrieved weather data in the respective columns
    sales_data.at[index, "Temperature_C"] = temp
    sales_data.at[index, "Humidity_%"] = humidity
    sales_data.at[index, "Weather_Description"] = description

    # Display the weather details for each row
    print(f"{temp:.1f}°C, Humidity: {humidity}%, Weather: {description}")



-0.5°C, Humidity: 37.9%, Weather: Clearing in the afternoon.
13.6°C, Humidity: 92.7%, Weather: Cloudy skies throughout the day with a chance of rain throughout the day.
-1.8°C, Humidity: 65.7%, Weather: Cloudy skies throughout the day with late afternoon rain or snow.
22.1°C, Humidity: 90.1%, Weather: Cloudy skies throughout the day with early morning rain.
1.6°C, Humidity: 81.8%, Weather: Cloudy skies throughout the day with a chance of rain or snow throughout the day.
-0.5°C, Humidity: 37.9%, Weather: Clearing in the afternoon.
13.6°C, Humidity: 92.7%, Weather: Cloudy skies throughout the day with a chance of rain throughout the day.
1.8°C, Humidity: 78.3%, Weather: Cloudy skies throughout the day with a chance of rain or snow throughout the day.
14.5°C, Humidity: 92.9%, Weather: Cloudy skies throughout the day with rain.
0.5°C, Humidity: 71.5%, Weather: Partly cloudy throughout the day with early morning snow or rain.
23.1°C, Humidity: 83.7%, Weather: Partly cloudy throughout the da

In [28]:
sales_data.head()

,date,product_id,sales_amount,store_location,Temperature_C,Humidity_%,Weather_Description
0,2025-02-05,P001,150,New York,-0.5,37.9,Clearing in the afternoon.
1,2025-02-05,P002,300,Los Angeles,13.6,92.7,Cloudy skies throughout the day with a chance ...
2,2025-02-05,P003,450,Chicago,-1.8,65.7,Cloudy skies throughout the day with late afte...
3,2025-02-05,P004,600,Houston,22.1,90.1,Cloudy skies throughout the day with early mor...
4,2025-02-05,P005,750,Seattle,1.6,81.8,Cloudy skies throughout the day with a chance ...


# Step 4: Load the Integrated Data into MongoDB


In [29]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.8 MB/s eta 0:00:00


In [32]:
import pandas as pd

# Load Sales Data
url = "https://raw.githubusercontent.com/DrManalJalloul/Introduction-to-Data-Engineering/main/sales_data.csv"
sales_data = pd.read_csv(url)

# Select the collection inside your MongoDB database
collection = db['SalesData']

# Convert DataFrame to a list of dictionaries
sales_data_dict = sales_data.to_dict(orient="records")

# Insert the data into MongoDB
collection.insert_many(sales_data_dict)

# Confirmation message
print("✅ Data successfully inserted into MongoDB!")

# Retrieve and display a few records for verification
for record in collection.find().limit(5):
    print(record)


✅ Data successfully inserted into MongoDB!
{'_id': ObjectId('67a7d5085180f8dd4cc3e3f2'), 'date': '2025-02-05', 'product_id': 'P001', 'sales_amount': 150, 'store_location': 'New York'}
{'_id': ObjectId('67a7d5085180f8dd4cc3e3f3'), 'date': '2025-02-05', 'product_id': 'P002', 'sales_amount': 300, 'store_location': 'Los Angeles'}
{'_id': ObjectId('67a7d5085180f8dd4cc3e3f4'), 'date': '2025-02-05', 'product_id': 'P003', 'sales_amount': 450, 'store_location': 'Chicago'}
{'_id': ObjectId('67a7d5085180f8dd4cc3e3f5'), 'date': '2025-02-05', 'product_id': 'P004', 'sales_amount': 600, 'store_location': 'Houston'}
{'_id': ObjectId('67a7d5085180f8dd4cc3e3f6'), 'date': '2025-02-05', 'product_id': 'P005', 'sales_amount': 750, 'store_location': 'Seattle'}
